In [1]:
import pandas as pd
import os
import sys

In [2]:
os.getcwd()

'/home/gunny/Github/fastText'

In [54]:
import numpy as np

In [50]:
lines_names = ["lineID",
                "characterID",
                "movieID",
                "character",
                "text"]

In [51]:
df_lines = pd.read_csv("data/movie_lines.tsv", delimiter="\t", header=None, names=lines_names)

In [52]:
df_lines.shape

(293368, 5)

In [64]:
df_lines.iloc[np.random.choice(df_lines.shape[0], 10)]["text"]

140643    Nice and easy now. Keep your hands where I can...
248740    You're a hell of a boy Frank. Someday you're g...
224486    Faith this guy you think is Mr. Right he's... ...
40850                       No doubt Paul thinks it stinks.
21048              Perhaps an hour perhaps a little longer.
171904                               That's what we agreed.
34289                                       Couldn't sleep.
13226       What has happened madam to annoy your ladyship?
285349                               I'm getting married...
199548                                       Oh no -- _ 14.
Name: text, dtype: object

In [65]:
titles_names = [
    "movieID", 
    "movie",
    "year", 
    "rating",
    "votes",
    "genres"
]

In [66]:
df_titles = pd.read_csv("data/movie_titles_metadata.tsv", delimiter="\t", header=None, names=titles_names)

In [67]:
df_titles.shape

(617, 6)

In [73]:
df_titles.head(5)

,movieID,movie,year,rating,votes,genres
0,m0,10 things i hate about you,1999,6.9,62847.0,['comedy' 'romance']
1,m1,1492: conquest of paradise,1992,6.2,10421.0,['adventure' 'biography' 'drama' 'history']
2,m2,15 minutes,2001,6.1,25854.0,['action' 'crime' 'drama' 'thriller']
3,m3,2001: a space odyssey,1968,8.4,163227.0,['adventure' 'mystery' 'sci-fi']
4,m4,48 hrs.,1982,6.9,22289.0,['action' 'comedy' 'crime' 'drama' 'thriller']


In [125]:
df_titles["genres_collection"] = df_titles["genres"].apply(lambda x: [genre[1:-1] for genre in str(x)[1:-1].split(" ")])

In [126]:
s = df_titles["genres_collection"].apply(pd.Series, 1).stack()

In [127]:
s.index = s.index.droplevel(-1)

In [128]:
s.head()

0       comedy
0      romance
1    adventure
1    biography
1        drama
dtype: object

In [129]:
s.name = "single_genre"

In [130]:
df_titlesbygenre = df_titles.join(s)

In [140]:
df_titlesbygenre.shape

(1870, 8)

In [135]:
popular_genres = df_titlesbygenre.groupby('single_genre')["movieID"].count().sort_values()
use_genres = popular_genres.index[-11:].tolist()

In [201]:
unwanted_genres = popular_genres.index[:popular_genres.shape[0]-11].tolist()

In [202]:
unwanted_genres

['adult',
 '',
 'documentary',
 'film-noir',
 'short',
 'sport',
 'musical',
 'western',
 'music',
 'family',
 'animation',
 'history',
 'war',
 'biography']

In [147]:
popular_genres

single_genre
adult            1
                 2
documentary      3
film-noir        4
short            5
sport            8
musical          8
western         12
music           13
family          17
animation       18
history         21
war             23
biography       25
fantasy         78
horror          99
mystery        101
adventure      116
sci-fi         120
romance        132
crime          146
comedy         162
action         168
thriller       269
drama          319
Name: movieID, dtype: int64

In [136]:
use_genres

['fantasy',
 'horror',
 'mystery',
 'adventure',
 'sci-fi',
 'romance',
 'crime',
 'comedy',
 'action',
 'thriller',
 'drama']

In [142]:
use_finder = df_titlesbygenre["single_genre"].apply(lambda x: x in use_genres)
df_usebygenre = df_titlesbygenre.loc[use_finder]

In [253]:
df_usebygenre.head()

,movieID,movie,year,rating,votes,genres,genres_collection,single_genre
0,m0,10 things i hate about you,1999,6.9,62847.0,['comedy' 'romance'],"[comedy, romance]",comedy
0,m0,10 things i hate about you,1999,6.9,62847.0,['comedy' 'romance'],"[comedy, romance]",romance
1,m1,1492: conquest of paradise,1992,6.2,10421.0,['adventure' 'biography' 'drama' 'history'],"[adventure, drama]",adventure
1,m1,1492: conquest of paradise,1992,6.2,10421.0,['adventure' 'biography' 'drama' 'history'],"[adventure, drama]",drama
2,m2,15 minutes,2001,6.1,25854.0,['action' 'crime' 'drama' 'thriller'],"[action, crime, drama, thriller]",action


In [143]:
all_genres = df_usebygenre["single_genre"].unique()

In [104]:
all_genres = np.delete(all_genres, np.where(np.logical_or(all_genres == 'a', all_genres == '')))

In [144]:
all_genres

array(['comedy', 'romance', 'adventure', 'drama', 'action', 'crime',
       'thriller', 'mystery', 'sci-fi', 'fantasy', 'horror'], dtype=object)

In [145]:
genre_map = {i: genre for i, genre in enumerate(all_genres)}

In [146]:
genre_map

{0: 'comedy',
 1: 'romance',
 2: 'adventure',
 3: 'drama',
 4: 'action',
 5: 'crime',
 6: 'thriller',
 7: 'mystery',
 8: 'sci-fi',
 9: 'fantasy',
 10: 'horror'}

In [234]:
rev_genre_map = {genre: i for i, genre in enumerate(all_genres)}
rev_genre_map

{'comedy': 0,
 'romance': 1,
 'adventure': 2,
 'drama': 3,
 'action': 4,
 'crime': 5,
 'thriller': 6,
 'mystery': 7,
 'sci-fi': 8,
 'fantasy': 9,
 'horror': 10}

In [150]:
df_linesbygenre = df_lines[["movieID", "text"]].merge(df_usebygenre[["movieID", "single_genre"]], on="movieID", how="inner")

In [164]:
# note: there are missing values for text
df_linesbygenre["text"].isnull().sum()

985

In [165]:
df_linesbygenre.groupby("single_genre").count().sort_values("text").sum()

movieID    787872
text       786887
dtype: int64

In [176]:
df_lines = df_lines.loc[df_lines["text"].notnull()]
df_lines.shape
# with missing text lines removed

(288823, 5)

In [171]:
df_lines.shape

(293368, 5)

## Training, testing set construction

,lineID,characterID,movieID,character,text
0,L1045,u0,m0,BIANCA,They do not!
1,L1044,u2,m0,CAMERON,They do to!
2,L985,u0,m0,BIANCA,I hope so.
3,L984,u2,m0,CAMERON,She okay?
4,L925,u0,m0,BIANCA,Let's go.


In [193]:
use_genres_set = set(use_genres)
genres_filter = df_titles["genres_collection"].apply(lambda x: True if len(set(x).difference(set(x).intersection(use_genres_set))) == 0 else False)

In [197]:
genres_filter.sum()
# not good to deselect film titles with ANY unwanted genre

492

In [195]:
df_titles.shape

(617, 8)

In [206]:
def safe_remove(x: list, unwanted: list):
    for elem in unwanted:
        if elem in x:
            x.remove(elem)
    return x

In [231]:
df_titles["wanted_genres"] = df_titles["genres_collection"].apply(lambda x: safe_remove(x, unwanted_genres))

In [232]:
df_titles["wanted_genres"].apply(lambda x: 1 if len(set(x).intersection(set(unwanted_genres))) > 0 else 0).sum()

0

In [246]:
df_titles["genres_label"] = df_titles["wanted_genres"].apply(lambda x: ",".join(["__label__" + str(rev_genre_map[genre]) for genre in x]))

In [247]:
df_set = df_titles[["genres_label", "movieID"]].merge(df_lines[["movieID", "text"]], on="movieID", how="inner")

In [248]:
del df_set["movieID"]

In [249]:
df_set.head()

,genres_label,text
0,"__label__0,__label__1",They do not!
1,"__label__0,__label__1",They do to!
2,"__label__0,__label__1",I hope so.
3,"__label__0,__label__1",She okay?
4,"__label__0,__label__1",Let's go.


In [250]:
df_set.shape

(288353, 2)

In [334]:
df_set["genres_list"] = df_set["genres_label"].apply(lambda x: x.split(","))
df_set["label"] = df_set["genres_list"].apply(lambda x: x[int(np.random.choice(len(x), 1))])

In [335]:
df_set.iloc[np.random.choice(df_set.shape[0], 10)]

,genres_label,text,genres_list,label
11928,"__label__4,__label__5,__label__3,__label__7,__...",Sorry I hit Jackson.,"[__label__4, __label__5, __label__3, __label__...",__label__3
123704,"__label__4,__label__6,__label__5",Yeah that's it. And he's like a fucking geniu...,"[__label__4, __label__6, __label__5]",__label__5
112555,__label__3,See anything interesting?,[__label__3],__label__3
163638,__label__0,Welcome to work Ed.,[__label__0],__label__0
199400,__label__3,We did not.,[__label__3],__label__3
254195,"__label__0,__label__3",I just started it.,"[__label__0, __label__3]",__label__0
40439,__label__3,She left you right?,[__label__3],__label__3
40985,"__label__0,__label__1",Scared? Swimming underwater with that oxygen d...,"[__label__0, __label__1]",__label__0
235854,"__label__5,__label__3,__label__7,__label__6",Hey check it out...,"[__label__5, __label__3, __label__7, __label__6]",__label__6
242881,"__label__0,__label__5",No. HEY -- HE KNOWS THAT YOU BEEN CREATING NOW...,"[__label__0, __label__5]",__label__0


In [336]:
df_set.groupby("label").count()

,genres_label,text,genres_list
label,,,
,1625,1625,1625
__label__0,38690,38690,38690
__label__1,28236,28236,28236
__label__10,12372,12372,12372
__label__2,12869,12869,12869
__label__3,80008,80008,80008
__label__4,19686,19686,19686
__label__5,21853,21853,21853
__label__6,38486,38486,38486


In [337]:
_df_set = df_set[["label", "text"]]

In [255]:
df_usebygenre["single_genre"].apply(lambda x: x not in use_genres).sum()

0

In [270]:
df_usebygenre["genres_label"] = df_usebygenre["single_genre"].apply(lambda x: "__label__" + str(rev_genre_map[x]))

/home/gunny/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [271]:
df_usebygenre.head()

,movieID,movie,year,rating,votes,genres,genres_collection,single_genre,genres_label
0,m0,10 things i hate about you,1999,6.9,62847.0,['comedy' 'romance'],"[comedy, romance]",comedy,__label__0
0,m0,10 things i hate about you,1999,6.9,62847.0,['comedy' 'romance'],"[comedy, romance]",romance,__label__1
1,m1,1492: conquest of paradise,1992,6.2,10421.0,['adventure' 'biography' 'drama' 'history'],"[adventure, drama]",adventure,__label__2
1,m1,1492: conquest of paradise,1992,6.2,10421.0,['adventure' 'biography' 'drama' 'history'],"[adventure, drama]",drama,__label__3
2,m2,15 minutes,2001,6.1,25854.0,['action' 'crime' 'drama' 'thriller'],"[action, crime, drama, thriller]",action,__label__4


In [272]:
df_set_unpivot = df_usebygenre[["genres_label", "movieID"]].merge(df_lines[["movieID", "text"]], on="movieID", how="inner")

In [273]:
del df_set_unpivot["movieID"]

In [274]:
df_set_unpivot.iloc[np.random.choice(df_set_unpivot.shape[0], 10)]

,genres_label,text
225910,__label__3,Good Lord!
371504,__label__7,What's your point Dude?
238940,__label__7,He's telling you the truth Edgar. Whoever it w...
335591,__label__1,That's amazing. I-I-I... I've been goin' for f...
387221,__label__7,Sir...
539448,__label__10,My vacation is over in a week.
287925,__label__9,Give me a hand with this sack of shit.
112259,__label__0,You're taking Mom out tonight?
317493,__label__3,No. I guess you're here for the backpack.
166456,__label__6,Why would anyone do something like this?


In [275]:
df_set_unpivot.shape

(786887, 2)

In [243]:
import csv

In [338]:
train_ratio = 0.8 
df = _df_set
train_idx = np.random.choice(df.shape[0], int(train_ratio * df.shape[0]), replace=False)
test_idx = np.delete(np.arange(df.shape[0]), train_idx)

df.iloc[train_idx].to_csv("data/movielines.train", header=None, index=False, sep="~")
df.iloc[test_idx].to_csv("data/movielines.test", header=None, index=False, sep="~")

In [228]:
np.delete(np.arange(df_set.shape[0]), train_idx).shape[0] + train_idx.shape[0]

288353

## Training Testing

In [286]:
import fastText

In [287]:
import os

In [288]:
train_data = "data/train.csv"
test_data = "data/test.csv"

## You just have to use the train and test data i uploaded & install fastText through `pip`

In [382]:
model = fastText.train_supervised(
    input=train_data, epoch=50, lr=1.0, wordNgrams=3, verbose=2, minCount=1, thread=4)

In [293]:
def print_results(N, p, r):
    print("N\t" + str(N))
    print("P@{}\t{:.3f}".format(1, p))
    print("R@{}\t{:.3f}".format(1, r))

In [376]:
print_results(*model.test(test_data))

N	57297
P@1	0.243
R@1	0.243


In [350]:
df_test = pd.read_csv(test_data, header=None, names=["label", "text"])

In [351]:
df_test.head()

,label,text
0,__label__4,it doesnt matter who you were before its who ...
1,__label__10,did you see jack ?
2,__label__2,the moons not down yet
3,__label__4,but it was a whole lot of killing why should ...
4,__label__8,so hows tricks ? sorry occupational humor


## Calculate accuracy using test set

In [349]:
results = []
for text in df_test["text"]:
    results.append(model.predict(text)[0][0])
hit = 0
for i, pred in enumerate(results):
    if pred == df_test["label"][i]:
        hit += 1
print(hit/df_test.shape[0])

(('__label__4',), array([0.97308511]))

## How to use this

In [364]:
genre_map[int(model.predict("who is the real tyler durden")[0][0][9:])]

'crime'

In [365]:
genre_map[int(model.predict("you shall not pass")[0][0][9:])]

'adventure'

In [367]:
genre_map[int(model.predict("i need a shower right now")[0][0][9:])]

'drama'

In [369]:
genre_map[int(model.predict("my name is ani and i am a yuppie")[0][0][9:])]

'drama'

In [363]:
genre_map[int(model.predict("the day abhijeet got his phd is the day nano-photons are declared to be physically translucent. science science science")[0][0][9:])]

'horror'

In [356]:
genre_map[int(model.predict("this bitch whose name i forgot gave me a hard time")[0][0][9:])]

'drama'

In [357]:
genre_map[int(model.predict("the white ")[0][0][9:])]

'drama'

In [ ]:
genre_map[int(model.predict("")[0][0][9:])]

In [381]:
del model

NameError: name 'model' is not defined